<a href="https://colab.research.google.com/github/Shyamalara/100-Days-Of-ML-Code/blob/master/datacleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
#import streamlit as st
#import sweetviz as sv
import torch

from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering, pipeline

In [4]:
# Suppress underflow warnings if they are non-critical
np.seterr(under='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

### Metholodogy

1. Started off with RJY as it had a variety of casing names, sizes and grades and  3 plants
2. Used Regex to separate out the Name, size and grade of the material
3. Used Hugging face model : tapas-base-finetuned-wtq. It is a table question answering tool
4. 26.3.2025 : go through the streamlit chatbot documentation and OpenAI course on open source models hugging face
5. 27.3.2025 - 28.3.2025 : Try and build the streamlit app

### Ideas
1. Using sentence similarity to cluster similar words
2. use visual question answering

 ### Reading the data

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
data_rjy = pd.read_excel("/content/drive/MyDrive/Tubular DATA- DS.xlsx", sheet_name = "RAJY")

### Data cleaning and preparation

In [10]:
data_rjy = data_rjy.drop(["Remarks"], axis = 1)

data_rjy = data_rjy.fillna(method = "ffill")

data_rjy.drop([33], axis = 0 )

data_rjy = data_rjy.applymap(
    lambda x: 0 if isinstance(x, (int, float)) and abs(x) < 1e-100 else x
)
data_rjy = data_rjy.fillna("Unknown")  # Replace NaNs with a placeholder

data_rjy = data_rjy.fillna("Unknown")  # Replace NaNs with a placeholder

data_rjy = data_rjy.rename(columns = {"Plant":"Plant code"})


<ipython-input-10-bf1a944d385e>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_rjy = data_rjy.fillna(method = "ffill")
<ipython-input-10-bf1a944d385e>:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_rjy = data_rjy.applymap(


In [11]:
def extract_values(description):
    name_match = re.search(r'\b(PREM CASING|CASING PIPE|DRILL PIPE|HEAVY WEIGHT DRILL PIPE|DRILL COLLAR SPIRAL|DRILL COLLAR|DC)\b',
                           description)  # Extract "PREM CASING"
    size_match = re.search(r'(\d+\s\d+/\d+\"|\d+\d*\"+|\d+\-+\d+/\d*\"|\d+\-+\d\/\d)', description)  # Extract "size""
    grade_match = re.search(r'([A-Z]-\d+)', description)  # Extract "grade"

    name = name_match.group(0) if name_match else None
    size = size_match.group(0) if size_match else None
    grade = grade_match.group(0) if grade_match else None

    return pd.Series([name, size, grade])

In [12]:

# Apply the function to extract values
data_rjy[['Name', 'Size', 'Grade']] = data_rjy['Mat description'].apply(extract_values)

In [14]:
data_rjy.head()

,Plant code,Plant description,EDP code ↑,X,Mat description,UoM,Stock Value in INR Crores as on 31.03.2024,Stock as on 31.03.2024,Pending PR Qtty. ( as on March 2024),Pending PO qtty.( as on March 2024),No Value Qtty.( as on March 2024 ),Net Stock Qtty. (=Total Stock +Pending PO Qtty. - No value Qtty.),Name,Size,Grade
0,41A1,"Rajahmundry Asset, Rajahmundry",20000074.0,2.0,"PREM CASING 7"", P-110, 29 PPF, TSH3SB (*",M,0.808270,1343.279,0.0,0.000,0,1343.279,PREM CASING,"7""",P-110
1,41A1,"Rajahmundry Asset, Rajahmundry",20000146.0,2.0,"PREM CASING 5"",P-110 18#,HSL-HC FLUSH JT",M,0.246883,995.912,0.0,0.588,0,996.500,PREM CASING,"5""",P-110
2,41A1,"Rajahmundry Asset, Rajahmundry",20000299.0,2.0,"CASING PIPE 7"" 29 PPF P-110 PREMIUM PRE*",M,0.279118,494.500,0.0,0.000,0,494.500,CASING PIPE,"7""",P-110
3,41A1,"Rajahmundry Asset, Rajahmundry",20003209.0,2.0,"CASING PIPE 7"" 29PPF L-80 BTC",M,0.109345,289.910,0.0,9.495,0,299.405,CASING PIPE,"7""",L-80
4,41A1,"Rajahmundry Asset, Rajahmundry",20003448.0,2.0,"CASING PIPE 13-3/8"" J-55 68PPF BTC",M,0.048607,79.250,0.0,0.000,0,79.250,CASING PIPE,"13-3/8""",J-55


In [15]:
# Convert all values to strings and create a dictionary in the proper format
data_rjy = data_rjy.astype(str)
data_rjy_dict = data_rjy.to_dict(orient="list")

In [16]:
# Load model & tokenizer
model = 'google/tapas-large-finetuned-wtq'
tapas_model = AutoModelForTableQuestionAnswering.from_pretrained(model)

# Convert model to float32 for numerical stability
tapas_model = tapas_model.to(torch.float32)

tapas_tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/262k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [ ]:
# model = 'google/tapas-base-finetuned-wtq'
# tapas_model = AutoModelForTableQuestionAnswering.from_pretrained(model)
# tapas_tokenizer = AutoTokenizer.from_pretrained(model)

In [17]:
# Initializing pipeline
nlp = pipeline('table-question-answering', model=tapas_model, tokenizer=tapas_tokenizer)


def qa(query,data):
    print('>>>>>')
    print(query)
    result = nlp({'table': data,'query':query})
    answer = result['cells']
    print(answer)

Device set to use cpu


In [18]:
prediction = qa("what is the Stock as on 31.03.2024 of EDP code 20000074.0 in plant 41A1 ", data_rjy)

>>>>>
what is the Stock as on 31.03.2024 of EDP code 20000074.0 in plant 41A1 


/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:2699: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.11/dist-packages/transformers/models/tapas/tokenization_tapas.py:1493: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


['1343.279']


In [21]:
prediction = qa("Tell me the plant code of Plant description Rajahmundry Asset, Rajahmundry", data_rjy_dict)

>>>>>
Tell me the plant code of Plant description Rajahmundry Asset, Rajahmundry
['41A1', '41A1', '41A1', '41A1', '41A1']


In [22]:
prediction = qa("what is the EDP code of PREM CASING, 7\", grade P-110", data_rjy_dict)

>>>>>
what is the EDP code of PREM CASING, 7", grade P-110
['20000074.0']
